In [1]:
import snakemake
from subprocess import run

# Modularity in Snakemake workflows

The Snakemake workflow engine has proven itself useful in creating, maintaining, and extending a variety of our bioinformatics analysis workflows.
We use Snakemake to create both single-purpose workflows as well as workflows that we expect to use and re-use frequently over an extended period of time.
Accordingly, we will be faced with two recurrent questions.

1. Which parts of which workflows can be re-used?
2. What is the best strategy for modularity and re-use?

We don't yet have a confident answer for those questions in the context of NBFAC workflows.
The purpose of this workshop is to describe three complementary strategies to creating workflows or workflow components that can be maintained in a single place and re-used in numerous contexts.

### Synopsis

1. **Includes** are typically a small set of re-usable rules that get embedded into a larger workflow and share its configuration. This is the simplest way to re-use workflow components.
2. **Subworkflows** allow one to invoke a distinct and self-contained workflow from another workflow. Each workflow can has its own configuration, working directory, and so on.
3. **Wrappers** are used at the rule level, and provide the smallest building block for constructing workflows from re-usable parts. Snakemake maintains a public repository of wrappers for common tools, but Snakemake can also make use of private or local wrapper definitions.

### Example workflow

![Workflow](workflow.png)

The demo workflow for this workshop is a fairly simple and linear workflow. In summary, it:

- Downsamples Illumina reads to a user-specified number of read pairs using seqtk
- Assembles the downsampled reads using SPAdes
- Maps the original reads back to the assembled contigs using Bowtie2
- Converts the alignment output in SAM format to BAM format, and sorts the aligned reads by position using SAMtools
- Calculates some summary statistics from the read mappings using SAMtools

The workflow consists of 9 rules: 3 related to proprocessing and assembly, 5 related to read mapping, and the 1 "default" rule to rule them all.
With subworkflows and includes, we can explore how to re-use groups of related rules.
With wrappers, we can explore how to replace rules running commonly used software with standardized invocation.

### Data

Before we get started, let's download some test data.

In [2]:
!./getdata.sh

Converting SRA file to Fastq format
Read 129850 spots for SRR5944233.1.sra
Written 129850 spots for SRR5944233.1.sra


And as a reference for our exercises, let's run the example Snakemake workflow using the Python API.

In [3]:
snakemake.snakemake(
    "Snakefile", configfiles=["config.json"], cores=4, targets=["all"],
    printshellcmds=True, workdir="sandbox/WD1/", dryrun=True
)

Building DAG of jobs...
Job counts:
	count	jobs
	1	all
	1	bam_stats
	1	copyseq
	1	downsample
	1	index_assembly
	1	index_bam
	1	map_back_reads
	1	map_sort
	1	spades
	9

[Fri Oct 16 00:33:29 2020]
rule copyseq:
    input: /home/jovyan/sample057-R1.fastq, /home/jovyan/sample057-R2.fastq
    output: seq/reads-R1.fastq, seq/reads-R2.fastq
    jobid: 6


        cp /home/jovyan/sample057-R1.fastq seq/reads-R1.fastq
        cp /home/jovyan/sample057-R2.fastq seq/reads-R2.fastq
        

[Fri Oct 16 00:33:29 2020]
rule downsample:
    input: seq/reads-R1.fastq, seq/reads-R2.fastq
    output: seq/reads-subset-R1.fastq, seq/reads-subset-R2.fastq
    jobid: 8


        seqtk sample -s137720190 seq/reads-R1.fastq 50000 > seq/reads-subset-R1.fastq
        seqtk sample -s137720190 seq/reads-R2.fastq 50000 > seq/reads-subset-R2.fastq
        

[Fri Oct 16 00:33:29 2020]
rule spades:
    input: seq/reads-subset-R1.fastq, seq/reads-subset-R2.fastq
    output: analysis/spades/scaffolds.fasta
    jobid: 

True

## Method 1: Includes

Snakemake's `include` statement allows a user to import the contents of another Snakefile.
This other file can implement an entire workflow itself, or represent only a fragment of a workflow.
Snakemake operates as if the user had copied and pasted the contents of the `include`d file into the main Snakefile (except that the default rule is not affected).
As a result, the main workflow and all included workflows share a single scope, and thus use a single shared configuration.

- pros
    - simplest to implement
    - included Snakefiles don't have to be free-standing workflows
- cons
    - shared scope encourages tight coupling of loosely related steps
    - shared config not easy to implement
    
### Exercise 1

For our first exercise, let's do the following.

1. Make a copy of `Snakefile` that contains only the preprocessing and assembly steps. We could call it `asmbl-inc.smk`.
2. Make a copy of `Snakefile` that contains only the mapping and postprocessing steps. We could call it `map-inc.smk`.
3. Make a very simple workflow that `include`s those two `.smk` files and implements a single rule. We could call it `Includes.smk`.

```python
include: "asmbl-inc.smk"
include: "map-inc.smk"

rule all:
    input:
        "analysis/sorted.bam.idxstats",
    run:
        print("Yay, all done!")
```

Then we test run the workflow with the following command.

In [ ]:
snakemake.snakemake(
    "Includes.smk", configfiles=["config.json"], cores=4, targets=["all"],
    printshellcmds=True, workdir="sandbox/WD2/", dryrun=True,
)

## Method 2: Subworkflows

Snakemake subworkflows allow one to invoke a distinct and self-contained workflow from another workflow.
Unlike includes, a subworkflow has a scope that is distinct from the Snakefile that calls it.
Subworkflows have their own distict configurations and working directories.

A subworkflow is imported and named using the `subworkflow` statement.

----------
```python
subworkflow foobar:
    snakefile: "foobar.smk"
    configfile: "foobar.json"
```
----------

If a rule in the main Snakefile depends on a file built by a subworkflow, simply wrap the filename in the subworkflow's name.

----------
```python
rule calc_summary:
    input: foobar("results.csv")
    output: "summary.dat"
    shell: "./summary.sh {input} > {output}"
```
----------

A Snakemake file can import any number of subworkflows.
However, if is not possible (as far as I can tell) to make one subworkflow depend on the output of another subworkflow.
This is A Shame, since that (in my opinion) is one of the most compelling use cases for subworkflows.

- pros
    - requires Snakemake files to be well-structured in a way that `include`s don't
    - distinct and isolated workflows are arguably better in a design sense
- cons
    - config file / workdir handling is a bit quirky
    - difficult to chain together

### Exercise 2

1. Make a copy of `Snakefile` that contains only the preprocessing and assembly steps. We could call it `asmbl-sub.smk`.
2. Make another copy of `Snakefile`. We could call it `Subworkflow.smk`.
    - Remove all of the preprocessing and assembly steps.
    - Import the subworkflow and name it something incredible clever like `assembly`.
    - For the rules in `Subworkflow.smk` that depend on files created by the `asmbl-sub.smk` subworkflow, make sure to wrap those filenames with the subworkflow name (see example below).

----------
```python
subworkflow assembly:
    snakefile: "asmbl-sub.smk"
    configfile: "asmbl.json"

rule index_reference:
    input:
        asmbl=assembly("analysis/spades/scaffolds.fasta")
    # ...and so on...
```
----------

Then we can test the workflow with the following command.

In [ ]:
snakemake.snakemake(
    "Subworkflow.smk", configfiles=["config.json"], cores=4, targets=["all"],
    printshellcmds=True, workdir="sandbox/WD3/", dryrun=True,
)

## Method 3: Wrappers

[Snakemake wrappers](https://snakemake-wrappers.readthedocs.io/en/stable/) provide a means to rapidly integrate popular bioinformatics tools into a workflow.
Unlike includes and subworkflows, wrappers operate at the level of individual rules, rather than groups of related rules.
And wrappers are not mutually exclusive with includes and subworkflows: it is possible to use wrappers in the main Snakemake file, any `include`d Snakemake files, and/or any subworkflows.

The Snakemake wrappers website lists the tools for which wrappers are available.
Each tool provides an example of how to invoke it, including labels for input and output files.
Rules using wrappers will look very similar to regular rules, except the `shell` or `run` block is replaced with a `wrapper` block that specifies the wrapper to be used.

----------
```python
rule downsample:
    input:
        f1="seq/reads-R1.fastq",
        f2="seq/reads-R2.fastq",
    output:
        f1="seq/reads-subset-R1.fastq.gz",
        f2="seq/reads-subset-R2.fastq.gz",
    params:
        n=config["sample_numreads"],
        seed=config["sample_seed"],
    wrapper:
        "0.64.0/bio/seqtk/subsample/pe"
```
----------

- pros
    - clean, concise rules
    - taking advantage of network effects of community conventions
- cons
    - may restrict flexibility
    - limited benefit in some cases

### Exercise 3

1. Make a copy of `Snakefile`. We could call it `Wrappers.smk`.
2. Refer to the Snakemake wrappers website to see which tools in the workflow already have wrappers. Use the examples listed on the website to update those rules and use the wrapper.

Then we can test the workflow with the following command.

In [ ]:
snakemake.snakemake(
    "Wrappers.smk", configfiles=["config.json"], cores=4, targets=["all"],
    printshellcmds=True, workdir="sandbox/WD4/", dryrun=True,
)